In [28]:
import requests
import pandas as pd 
import numpy as np
import os
import re
import datetime as dt
from bs4 import BeautifulSoup

In [11]:
def get_year_urls() -> list: 
  """
  Retrieves the URLs for every year's page in the USCRN index.
  
  Returns:
  year_urls (list): A list of URLs for every year's page.
  """

  url = "https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/"
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  links = soup.find_all("a") 
  years = [str(x).zfill(1) for x in range(2000,2024)]
  year_urls = [url + link['href'] for link in links if link['href'].rstrip('/') in years]
  return year_urls

def get_file_urls() -> list: 
  """
  Retrieves the URLs for every file contained on each year's page.

  Returns: 
  file_urls (list): A list of file URLs.
  """
  year_urls = get_year_urls()

  file_urls = []
  for url in year_urls: 
    response = requests.get(url) 
    soup = BeautifulSoup(response.content, 'html.parser')
    file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
    if file_links:
      new_file_urls = [url + link.getText() for link in file_links]
      file_urls.extend(new_file_urls)
  return file_urls


def get_station_location(url) -> str: 
  """
  Extracts the name of the station from a given URL.
  
  Args:
  url (str): The URL to extract the station name from.
  
  Returns:
  station_location (str): The name of the station.
  """
  regex = r"([St.]*[A-Z][a-z]+_*[A-Za-z]*).*.txt" 
  file_name = re.search(regex, url).group(0)
  station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
  return  station_location
  

# Get rows for current batch
url = "https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Aleknagik_1_NNE.txt"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [12]:
#Get location from url
station_location = get_station_location(url)
print(station_location)
# Get new rows 
rows = [re.split('\s+', row) for row in str(soup).strip().split("\n")]
print(len(rows[0]))

Aleknagik
23


In [14]:

wind_cols = [[station_location] + row[:5] + row[-2:] for row in rows]
print(len(wind_cols[0]))


8


In [15]:
columns = ['station_location','wbanno','utc_date','utc_time',
'lst_date','lst_time',"wind_1_5", "wind_flag"]

In [27]:
df = pd.DataFrame(wind_cols, columns=columns)

# # convert wind_1_5 to float -- replace any missing values with nano
# df['wind_1_5'] = df['wind_1_5'].astype(float)
# df = df[df['wind_1_5'] >= 0]

# # convert to datetimes
# df['utc_datetime'] = pd.to_datetime(df['utc_date'].astype(int).astype(str) + df['utc_time'].astype(int).astype(str).str.zfill(4), format='%Y%m%d%H%M')
# df['lst_datetime'] = pd.to_datetime(df['lst_date'].astype(int).astype(str) + df['lst_time'].astype(int).astype(str).str.zfill(4), format='%Y%m%d%H%M')

# # round to nearest hour 
# df['utc_datetime'] = df['utc_datetime'].dt.floor("H")
# df['lst_datetime'] = df['lst_datetime'].dt.floor("H")

# # calculate hourly averages 
# df = df.groupby(['station_location','wbanno','utc_datetime','lst_datetime','wind_flag'])['wind_1_5'].mean().reset_index()

# # rename wind column 
# df.rename({"wind_1_5":"wind_hr_avg"}, axis=1, inplace=True)

# df

In [28]:
df

,station_location,wbanno,utc_date,utc_time,lst_date,lst_time,wind_1_5,wind_flag
0,Aleknagik,23583,20230101,0005,20221231,1505,3.09,0
1,Aleknagik,23583,20230101,0010,20221231,1510,2.96,0
2,Aleknagik,23583,20230101,0015,20221231,1515,2.70,0
3,Aleknagik,23583,20230101,0020,20221231,1520,2.84,0
4,Aleknagik,23583,20230101,0025,20221231,1525,2.76,0
...,...,...,...,...,...,...,...,...
17767,Aleknagik,23583,20230303,1640,20230303,0740,0.31,0
17768,Aleknagik,23583,20230303,1645,20230303,0745,0.62,0
17769,Aleknagik,23583,20230303,1650,20230303,0750,0.75,0
17770,Aleknagik,23583,20230303,1655,20230303,0755,0.63,0


In [26]:
def transform_dataframe(df) -> pd.DataFrame:
  """
  Transforms a Pandas DataFrame created from a list of lists in process_rows().
    
  Args:
  df (pandas.DataFrame): The DataFrame to be transformed.
  
  Returns:
  transformed_df (pandas.DataFrame): The transformed DataFrame.
  """
  df = df.copy()

  # convert wind_1_5 to float -- drop any negative measurements
  df['wind_1_5'] = df['wind_1_5'].astype(float)
  df = df[df['wind_1_5'] >= 0]

  # convert to datetimes
  df['utc_datetime'] = pd.to_datetime(df['utc_date'].astype(int).astype(str) + df['utc_time'].astype(int).astype(str).str.zfill(4), format='%Y%m%d%H%M')
  df['lst_datetime'] = pd.to_datetime(df['lst_date'].astype(int).astype(str) + df['lst_time'].astype(int).astype(str).str.zfill(4), format='%Y%m%d%H%M')

  # round to nearest hour 
  df['utc_datetime'] = df['utc_datetime'].dt.floor("H")
  df['lst_datetime'] = df['lst_datetime'].dt.floor("H")

  # calculate hourly averages 
  df = df.groupby(['station_location','wbanno','utc_datetime','lst_datetime','wind_flag'])['wind_1_5'].mean().reset_index()

  # rename wind column 
  df.rename({"wind_1_5":"wind_hr_avg"}, axis=1, inplace=True)

  return df 


In [29]:
df2 = transform_dataframe(df)

df2

,station_location,wbanno,utc_datetime,lst_datetime,wind_flag,wind_hr_avg
0,Aleknagik,23583,2023-01-01 00:00:00,2022-12-31 15:00:00,0,2.846364
1,Aleknagik,23583,2023-01-01 01:00:00,2022-12-31 16:00:00,0,3.035833
2,Aleknagik,23583,2023-01-01 02:00:00,2022-12-31 17:00:00,0,2.978333
3,Aleknagik,23583,2023-01-01 03:00:00,2022-12-31 18:00:00,0,2.718333
4,Aleknagik,23583,2023-01-01 04:00:00,2022-12-31 19:00:00,0,2.882500
...,...,...,...,...,...,...
1477,Aleknagik,23583,2023-03-03 13:00:00,2023-03-03 04:00:00,0,0.546667
1478,Aleknagik,23583,2023-03-03 14:00:00,2023-03-03 05:00:00,0,0.512500
1479,Aleknagik,23583,2023-03-03 15:00:00,2023-03-03 06:00:00,0,0.565000
1480,Aleknagik,23583,2023-03-03 16:00:00,2023-03-03 07:00:00,0,0.534167


In [123]:
df[df['wind_hr_avg'] >= 0]

,station_location,wbanno,utc_datetime,lst_datetime,wind_flag,wind_hr_avg
0,Aleknagik,23583,2023-01-01 00:00:00,2022-12-31 15:00:00,0,2.846364
1,Aleknagik,23583,2023-01-01 01:00:00,2022-12-31 16:00:00,0,3.035833
2,Aleknagik,23583,2023-01-01 02:00:00,2022-12-31 17:00:00,0,2.978333
3,Aleknagik,23583,2023-01-01 03:00:00,2022-12-31 18:00:00,0,2.718333
4,Aleknagik,23583,2023-01-01 04:00:00,2022-12-31 19:00:00,0,2.882500
...,...,...,...,...,...,...
1475,Aleknagik,23583,2023-03-03 11:00:00,2023-03-03 02:00:00,0,0.607500
1476,Aleknagik,23583,2023-03-03 12:00:00,2023-03-03 03:00:00,0,0.558333
1477,Aleknagik,23583,2023-03-03 13:00:00,2023-03-03 04:00:00,0,0.546667
1478,Aleknagik,23583,2023-03-03 14:00:00,2023-03-03 05:00:00,0,0.512500


In [93]:
df2[df2['utc_datetime'] == pd.Timestamp("2023-01-01 00:00:00")]

,station_location,wbanno,utc_datetime,lst_datetime,wind_flag,wind_1_5
0,Aleknagik,23583,2023-01-01,2022-12-31 15:05:00,0,3.09
1,Aleknagik,23583,2023-01-01,2022-12-31 15:10:00,0,2.96
2,Aleknagik,23583,2023-01-01,2022-12-31 15:15:00,0,2.70
3,Aleknagik,23583,2023-01-01,2022-12-31 15:20:00,0,2.84
4,Aleknagik,23583,2023-01-01,2022-12-31 15:25:00,0,2.76
5,Aleknagik,23583,2023-01-01,2022-12-31 15:30:00,0,2.80


In [100]:
df2['utc_datetime'][0]

pd.Timestamp("2022-12-31 15:05:00").time().hour

15

In [30]:
import requests
import pandas as pd 
import numpy as np
import os
import re
from datetime import datetime
from bs4 import BeautifulSoup
def get_year_urls() -> list: 
  """
  Retrieves the URLs for every year's page in the USCRN index.
  
  Returns:
  year_urls (list): A list of URLs for every year's page.
  """

  url = "https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/"
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  links = soup.find_all("a") 
  years = [str(x).zfill(1) for x in range(2000,2024)]
  year_urls = [url + link['href'] for link in links if link['href'].rstrip('/') in years]
  return year_urls

def get_file_urls() -> list: 
  """
  Retrieves the URLs for every file contained on each year's page.

  Returns: 
  file_urls (list): A list of file URLs.
  """
  year_urls = get_year_urls()

  file_urls = []
  for url in year_urls: 
    response = requests.get(url) 
    soup = BeautifulSoup(response.content, 'html.parser')
    file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
    if file_links:
      new_file_urls = [url + link.getText() for link in file_links]
      file_urls.extend(new_file_urls)
  return file_urls


def get_station_location(url) -> str: 
  """
  Extracts the name of the station from a given URL.
  
  Args:
  url (str): The URL to extract the station name from.
  
  Returns:
  station_location (str): The name of the station.
  """
  regex = r"([St.]*[A-Z][a-z]+_*[A-Za-z]*).*.txt" 
  file_name = re.search(regex, url).group(0)
  station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
  return  station_location

def process_rows(file_urls, row_limit, output_file) -> None:
  """
  Processes a batch of rows from a list of URLs to extract weather station data and save it to a CSV file.

  Args:
    file_urls (list): A list of URLs where weather station data can be found.
    row_limit (int): The maximum number of rows to process per batch.
    output_file (str): The path to the output CSV file.
  Returns:
    None
  """
  # Get rows for current batch
  rows = []
  current_idx=0
  for i, url in enumerate(file_urls[current_idx:]):
    # Get location from url
    station_location = get_station_location(url)
    # Get new rows 
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    lines = [re.split('\s+', line) for line in str(soup).strip().split("\n")]
    wind_cols = [[station_location] + line[:5] + line[-2:] for line in lines]
    # Add to list
    rows.extend(wind_cols)
    if len(rows) >= row_limit:
      current_idx=i
      break

  # Define column names
  columns = ['station_location','wbanno','utc_date','utc_time',
  'lst_date','lst_time',"wind_1_5", "wind_flag"]
  
  # Create dataframe for current batch
  df = pd.DataFrame(rows, columns=columns)

  ####  Transform dataframe  #### 

  # convert wind_1_5 to float -- drop any negative measurements
  df['wind_1_5'] = df['wind_1_5'].astype(float)

  # convert to datetimes
  df['utc_datetime'] = pd.to_datetime(df['utc_date'].astype(int).astype(str) + df['utc_time'].astype(int).astype(str).str.zfill(4), format='%Y%m%d%H%M')
  df['lst_datetime'] = pd.to_datetime(df['lst_date'].astype(int).astype(str) + df['lst_time'].astype(int).astype(str).str.zfill(4), format='%Y%m%d%H%M')

  # round to nearest hour 
  df['utc_datetime'] = df['utc_datetime'].dt.floor("H")
  df['lst_datetime'] = df['lst_datetime'].dt.floor("H")

  # calculate hourly averages 
  df = df.groupby(['station_location','wbanno','utc_datetime','lst_datetime','wind_flag'])['wind_1_5'].mean().reset_index()

  # rename wind column 
  df.rename({"wind_1_5":"wind_hr_avg"}, axis=1, inplace=True)

  #### -------------- #####

  # Write dataframe to CSV
  if os.path.isfile(output_file):
      df.to_csv(output_file, mode='a', header=False, index=False)
  else:
    with open(output_file, "w") as fp:
      df.to_csv(fp, index=False)
  
  # Recursively process remaining rows     
  if len(rows) >= row_limit:
      remaining_urls = file_urls[current_idx:]
      process_rows(remaining_urls, row_limit, output_file)
  else: 
      return 


In [33]:
file_urls = ['https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2006/CRNS0101-05-2006-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2006/CRNS0101-05-2006-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2006/CRNS0101-05-2006-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2006/CRNS0101-05-2006-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2007/CRNS0101-05-2007-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2007/CRNS0101-05-2007-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2007/CRNS0101-05-2007-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2007/CRNS0101-05-2007-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2008/CRNS0101-05-2008-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2008/CRNS0101-05-2008-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2008/CRNS0101-05-2008-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2008/CRNS0101-05-2008-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2009/CRNS0101-05-2009-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2010/CRNS0101-05-2010-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2011/CRNS0101-05-2011-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2012/CRNS0101-05-2012-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2013/CRNS0101-05-2013-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2014/CRNS0101-05-2014-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2015/CRNS0101-05-2015-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2016/CRNS0101-05-2016-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2017/CRNS0101-05-2017-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2018/CRNS0101-05-2018-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2019/CRNS0101-05-2019-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2020/CRNS0101-05-2020-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2021/CRNS0101-05-2021-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2022/CRNS0101-05-2022-AK_Yakutat_3_SSE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Aleknagik_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Bethel_87_WNW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Cordova_14_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Deadhorse_3_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Denali_27_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Fairbanks_11_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Glennallen_64_N.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Gustavus_2_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Ivotuk_1_NNE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Kenai_29_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_King_Salmon_42_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Metlakatla_6_S.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Port_Alsworth_1_SW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Red_Dog_Mine_3_SSW.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Ruby_44_ESE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Sand_Point_1_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Selawik_28_E.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Sitka_1_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_St._Paul_4_NE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Tok_70_SE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Toolik_Lake_5_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Utqiagvik_formerly_Barrow_4_ENE.txt',
 'https://www.ncei.noaa.gov/pub/data/uscrn/products/subhourly01/2023/CRNS0101-05-2023-AK_Yakutat_3_SSE.txt']

In [34]:
process_rows(file_urls=file_urls, row_limit=300000, output_file="../data/uscrn_wind.csv")

: 

: 